In [1]:
from imutils.object_detection import non_max_suppression
from sklearn.svm import *
from skimage.transform import pyramid_gaussian
from skimage import color
import imutils
import os
import sys
import glob
import joblib
from google.colab.patches import cv2_imshow
import pickle
import itertools
import glob
import os.path
from PIL import Image
import numpy as np
import cv2

In [ ]:
sys.path.append("/content/")
from utils import *
from skimage.feature import hog

orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
threshold = .3

X,Y = make_training_data(orientations,pixels_per_cell,cells_per_block,threshold)

In [ ]:
print('Saving the classifier')
svm = SVC(kernel='rbf', gamma='auto', C= 0.1, max_iter=200000, tol=1e-4, coef0=1)
svm.fit(X, Y)
joblib.dump(svm, 'model_svm.npy')

In [ ]:
from skimage.feature import hog

orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
threshold = .3

def sliding_window(image, stepSize, windowSize):
  for y in range(0, image.shape[0], stepSize):
    for x in range(0, image.shape[1], stepSize):
      yield (x, y, image[y: y + windowSize[1], x:x + windowSize[0]])

model = joblib.load("/content/model_svm.npy")

scale = 0
detections = []

img= cv2.imread("/content/drive/MyDrive/CV/seq03-img-left/image_00000005_0.png")

# img= cv2.resize(img,(400,300)) # can change the size to default by commenting this code out our put in a random number
# img = imutils.resize(img,width=min(400, img.shape[1]))

(winW, winH)= (64,128)
windowSize=(winW,winH)
downscale=1.5
# Apply sliding window:
for resized in pyramid_gaussian(img, downscale=1.3): # loop over each image of the pyramid
    for (x,y,window) in sliding_window(resized, stepSize=10, windowSize=(winW,winH)):
        
        if window.shape[0] != winH or window.shape[1] !=winW : 
            continue
        window=color.rgb2gray(window)
        fds = hog(window, orientations, pixels_per_cell, cells_per_block, block_norm='L2')  
        fds = fds.reshape(1, -1) 
        pred = model.predict(fds) 
        
        if pred == 1:
            if model.decision_function(fds) > 1.3 and model.decision_function(fds) < 1.4 and scale<3 and scale>=1:  
            # if model.decision_function(fds) > 1.05 and model.decision_function(fds) < 1.1:  # set a threshold value for the SVM prediction i.e. only firm the predictions above probability of 0.6
                # print("Detection:: Location -> ({}, {})".format(x, y))
                # print("Scale ->  {} | Confidence Score {} \n".format(scale,model.decision_function(fds)))
                detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), model.decision_function(fds),
                                   int(windowSize[0]*(downscale**scale)),
                                      int(windowSize[1]*(downscale**scale))))
    scale+=1
    
clone = resized.copy()
# for (x_tl, y_tl, _, w, h) in detections:
#     cv2.rectangle(img, (x_tl, y_tl), (x_tl + w, y_tl + h), (0, 0, 255), thickness = 2)
rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections]) # do nms on the detected bounding boxes
sc = [score[0] for (x, y, score, w, h) in detections]
print("detection confidence score: ", sc)
sc = np.array(sc)
pick = non_max_suppression(rects, probs = sc, overlapThresh = 0.3)

        
for (xA, yA, xB, yB) in pick:
    cv2.rectangle(img, (xA, yA), (xB, yB), (0,255,0), 2)
cv2_imshow(img)